In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import cluster
import os
import re
# import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
os.getcwd()


'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play1'

In [3]:
df = pd.read_csv('./ou_daily_kpi.csv')
# df.dropna(axis = 1, inplace = True, how = 'all')
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
# df.to_csv('./dws_dsc_wh_ou_daily_kpi_july.csv', encoding='utf_8_sig')
df = df.dropna(axis =0,subset =['ou_code'])
# df.to_csv('./daily_kpi_all_810.csv', encoding='utf_8_sig')

In [5]:
# re1 = re.compile(r'(?<=\.).+')
# df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
df['operation_day'].apply(str).apply(int).min()

20210406

In [1]:
 
# clean_df2 = df.groupby('ou_code')[[
#     'inbound_receive_qty', 'outbound_shipped_qty'
#     ]].sum().reset_index()
# clean_df2['sum'] = clean_df2.sum(axis = 1)
# clean_df2 = clean_df2[clean_df2['sum'] != 0]
# df = df[df['ou_code'].isin(clean_df2.ou_code)]

In [20]:

clean_df1 = (df.groupby('ou_code')['operation_day'].count() <2).reset_index()
clean_df1.columns = ['ou_code', 'flag1']
df = clean_df1.merge(df, on = 'ou_code', how = 'inner')
df = df[df['flag1'] == False]

clean_df2 = df.groupby('ou_code')[[
    'inbound_receive_qty', 'outbound_shipped_qty'
    ]].sum().reset_index()
clean_df2['sum'] = clean_df2.sum(axis = 1)
clean_df2 = clean_df2[clean_df2['sum'] != 0]
df = df[df['ou_code'].isin(clean_df2.ou_code)]

clean_df3 = (df.groupby('ou_code')[[
    'total_working_hour'
    ]].sum() == 0).reset_index()
clean_df3 = clean_df3[clean_df3['total_working_hour'] == False]


# clean_df4 = (df.groupby('ou_code')[[
#     'outsource_working_hour'
#     ]].sum() == 0).reset_index()
# clean_df4 = clean_df4[clean_df4['outsource_working_hour'] == False]
# df = df[df['ou_code'].isin(clean_df4.ou_code)]
# df= df.reset_index()
 
# df = df[df['ou_code'].isin(clean_df3.ou_code)]
# df= df.reset_index()

df = df[[
    'ou_code','operation_day', 'inbound_receive_qty', 'is_holiday',
    'outbound_shipped_qty','total_head_count','total_working_hour',
    'outsource_working_hour', 'perm_working_hour',
    'other_working_hour', 'direct_working_hour', 'indirect_working_hour',
    'outbound_inbound_qty_ratio', 'perm_working_hour_ratio',
    'working_hour_per_head', 'location_usage_rate', 'location_idle_rate']]
df = df.fillna(0)
df = df[df['total_working_hour'] != 0]
df.head()

,ou_code,operation_day,inbound_receive_qty,is_holiday,outbound_shipped_qty,total_head_count,total_working_hour,outsource_working_hour,perm_working_hour,other_working_hour,direct_working_hour,indirect_working_hour,outbound_inbound_qty_ratio,perm_working_hour_ratio,working_hour_per_head,location_usage_rate,location_idle_rate
483,ABBOTBJXWS,20200408,0.0,0,0.0,3.0,24.50,0.0,24.50,0.0,0.0,24.50,0.000000,1.0,8.166667,0.0,0.0
484,ABBOTBJXWS,20201111,750.0,0,5986.0,4.0,32.30,0.0,32.30,0.0,0.0,32.30,7.981333,1.0,8.075000,0.0,0.0
486,ABBOTBJXWS,20200316,0.0,0,0.0,4.0,36.00,0.0,36.00,0.0,0.0,36.00,0.000000,1.0,9.000000,0.0,0.0
487,ABBOTBJXWS,20200624,0.0,0,0.0,3.0,25.00,0.0,25.00,0.0,0.0,25.00,0.000000,1.0,8.333333,0.0,0.0
488,ABBOTBJXWS,20200915,0.0,0,0.0,4.0,32.63,0.0,32.63,0.0,0.0,32.63,0.000000,1.0,8.157500,0.0,0.0


In [9]:
# view = df[df['ou_code'] == 'CN-066'].sort_values('operation_day')

In [ ]:
len(df['ou_code'].unique())

In [7]:
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans 

In [ ]:
# from sklearn.metrics import davies_bouldin_scores
daylen = int(df['operation_day'].nunique()/10)
def mnb_kmeans_in(ou_code):
        """
        mini batch kmeans, inbound, outbound, working hour data.
        simple algorithm, adding cols {max, min, mean, median, 75 quantile, distance to kernal}
        Calculate Davies Bouldin score
        from sklearn.metrics import davies_bouldin_score
        null data fill
        """

        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'inbound_receive_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
        df_fin['kernal_core1' ] = -1
        df_fin['kernal_value1'] = 0
        df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      

        """
        auto choose kernel nums
        """
        scores = []
        for center in list(range(2,daylen)):
            kmeans = cluster.MiniBatchKMeans(n_clusters=center)
            model  = kmeans.fit_predict(
                        X = np.reshape(list(df_rec['inbound_receive_qty']), (-1,1))
                        )
            score  = davies_bouldin_score(
                        np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)), model
                        )
            scores.append(score)

        center = np.argmin(scores)
        print(center)
     
        """
        not null data training
        """
    
        alg1 = cluster.MiniBatchKMeans(n_clusters = center, random_state = 5290403)
        hist1 = alg1.fit(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
        df_rec['kernal_core1'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,center))], axis = 1
                )
        
        cl_1.columns = ['kernal_value1', 'kernal_core1']
        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core1', how = 'inner'
                )
        
        """
        merging
        """
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)

        # df_fin['kind'] = 'inbound'

        return df_fin



def mnb_kmeans_out(ou_code):
        
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outbound_shipped_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['outbound_shipped_qty'] == 0])
        df_fin['kernal_core2' ] = -1
        df_fin['kernal_value2'] = 0
        df_rec = df_sub[df_sub['outbound_shipped_qty'] != 0]

        scores = []
        for center in list(range(2,daylen)):
            kmeans = cluster.MiniBatchKMeans(n_clusters=center)
            model  = kmeans.fit_predict(
                        X = np.reshape(list(df_rec['outbound_shipped_qty']), (-1,1))
                        )
            score  = davies_bouldin_score(
                        np.reshape(list(df_rec['outbound_shipped_qty']), (-1,1)), model
                        )
            scores.append(score)

        center = np.argmin(scores)
        print(center)
        alg1 = cluster.MiniBatchKMeans(n_clusters = center, random_state = 5290403)
        hist1 = alg1.fit(np.reshape(list(df_rec['outbound_shipped_qty']), (-1,1)))

        df_rec['kernal_core2'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,center))], axis = 1
                )
        
        cl_1.columns = ['kernal_value2', 'kernal_core2']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core2', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        # df_fin['kind'] = 'outbound'
        return df_fin

 
 

In [ ]:
mnb_kmeans_in('CN-214')

In [ ]:
# df_fin = pd.DataFrame()
# df_sub = df[df['ou_code'] == 'CN-305'][['ou_code', 'operation_day', 'inbound_receive_qty']]        
# df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
# df_fin['kernal_core1' ] = -1
# df_fin['kernal_value1'] = 0
# """
# not null data training
# """
# df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
# scores = []
# for center in list(range(2,10)):
#     kmeans = cluster.KMeans(n_clusters=center)
#     model  = kmeans.fit_predict(X = np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
#     score  = davies_bouldin_score(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)), model)
#     scores.append(score)
# center = min(scores)
# np.argmin(scores)

In [11]:

def mnb_kmeans_in(ou_code):
        """
        mini batch kmeans, inbound, outbound, working hour data.
        simple algorithm, adding cols {max, min, mean, median, 75 quantile, distance to kernal}
        """
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        """
        null data fill
        """
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'inbound_receive_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
        df_fin['kernal_core1' ] = -1
        df_fin['kernal_value1'] = 0
        """
        not null data training
        """
        df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
        hist1 = alg1.fit(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
        df_rec['kernal_core1'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value1', 'kernal_core1']
        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core1', how = 'inner'
                )
        
        """
        merging
        """
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)

        # df_fin['kind'] = 'inbound'

        return df_fin


def mnb_kmeans_out(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outbound_shipped_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['outbound_shipped_qty'] == 0])
        df_fin['kernal_core2' ] = -1
        df_fin['kernal_value2'] = 0
        df_rec = df_sub[df_sub['outbound_shipped_qty'] != 0]

        hist1 = alg1.fit(np.reshape(list(df_rec['outbound_shipped_qty']), (-1,1)))

        df_rec['kernal_core2'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value2', 'kernal_core2']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core2', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        # df_fin['kind'] = 'outbound'
        return df_fin

def mnb_kmeans_hr(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'total_working_hour']]        
        df_fin = df_fin.append(df_sub[df_sub['total_working_hour'] == 0])
        df_fin['kernal_core3' ] = -1
        df_fin['kernal_value3'] = 0
        df_rec = df_sub[df_sub['total_working_hour'] != 0]

        hist1 = alg1.fit(np.reshape(list(df_rec['total_working_hour']), (-1,1)))

        df_rec['kernal_core3'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value3', 'kernal_core3']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core3', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        """
        组内kernal distance 
        """
        df_fin['dis_core']=df_fin.groupby(
                'kernal_core3'
                )['total_working_hour','kernal_value3'].diff(
                        axis = 1
                        ).drop('total_working_hour', axis = 1).round(3)
        
        return df_fin



In [12]:
len(df['ou_code'].unique())


65

In [13]:
%%time
ou_codes = list(df['ou_code'].unique())
p = list()
for i in ou_codes:
    try: 
        mnb_kmeans_in(i)
        mnb_kmeans_out(i)
        mnb_kmeans_hr(i)
        # mnb_kmeans_hr2(i)
    except:
        
        p.append(i)


for i in p:
    ou_codes.remove(i)
[p, len(p), len(ou_codes)]

Wall time: 18.7 s


[['CN-070', 'CN-392', 'HK-015'], 3, 62]

In [14]:
%%time
from functools import reduce
df_final = pd.DataFrame()
for i in ou_codes:
        df_final = df_final.append(
            reduce(
                lambda left,right: pd.merge(
                    left,right,on= ['ou_code', 'operation_day']
                ), [mnb_kmeans_in(i), mnb_kmeans_out(i), mnb_kmeans_hr(i)]
                )
    )

"""
for loop , 对所有ou进行独立的kmeans on inb qty and outb qty
随后merge 原始表
"""

# np.setdiff1d(ou_codes,df_final.ou_code.unique())
df_final = df_final.merge(
    df[['ou_code','operation_day',\
        'outbound_inbound_qty_ratio','working_hour_per_head','total_head_count','is_holiday']],
    on = ['ou_code', 'operation_day'],
    how = 'left'
    )


Wall time: 18.1 s


In [15]:
# df_final.groupby(
#                 'kernal_core3'
#                 )['total_working_hour','kernal_value3'].diff(axis = 1).drop('total_working_hour', axis = 1)

In [16]:
# df_final['outsource_working_hour']

In [17]:
def mnb_kmeans_hr2(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outsource_working_hour']]  
        df_fin = df_fin.append(df_sub[df_sub['outsource_working_hour'] == 0])
        df_fin['kernal_core4' ] = -1
        df_fin['kernal_value4'] = 0
        df_rec = df_sub[df_sub['outsource_working_hour'] != 0]
 
        hist1 = alg1.fit(np.reshape(list(df_rec['outsource_working_hour']), (-1,1)))

        df_rec['kernal_core4'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value4', 'kernal_core4']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core4', how = 'outer'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        
        df_fin['dis_core_os']=df_fin.groupby(
                'kernal_core4'
                )['outsource_working_hour','kernal_value4'].diff(
                        axis = 1
                        ).drop('outsource_working_hour', axis = 1).round(3)
        return df_fin


ou_codes
p = list()
for i in ou_codes:
    try: 
        # mnb_kmeans_in(i)
        # mnb_kmeans_out(i)
        # mnb_kmeans_hr(i)
        mnb_kmeans_hr2(i)
    except:
        
        p.append(i)


for i in p:
    ou_codes.remove(i)
[len(p),len(ou_codes)]

[29, 33]

In [18]:
df_final2 = pd.DataFrame()
for i in ou_codes:
        df_final2 = df_final2.append(
            mnb_kmeans_hr2(i)
        )

df_final = df_final.merge(df_final2, on =  ['ou_code', 'operation_day'], how = 'left').fillna(0)

In [29]:
# model = sm.OLS(Y = X)
# results = model.fit()
# results.params
view = df_final[df_final['ou_code'] == 'CN-066'].sort_values('operation_day')

In [ ]:
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px

from datetime import datetime
import time

plt = df_final[df_final['qt_75_wh']!=0]
plt['operation_day'] = [datetime.strptime(i,'%Y%m%d').strftime('%Y%m%d') for i in plt['operation_day'].astype(str)]
plt['operation_day']= pd.to_datetime(plt['operation_day'])

In [ ]:
# # # sns.barplot(x = 'operation_day', y = ['inbound_receive_qty', 'outbound_shipped_qty'],  \
# # #     data=df_final.iloc[0:30,:], color = 'red'
# # #     ) 
# # # # g.set(xlim=(1, 20210805))  
# g = px.bar(plt[plt['ou_code'] == 'CN-298'],\
#     x = 'operation_day', y = ['inbound_receive_qty', 'outbound_shipped_qty']);g


In [ ]:
plt2 = plt[plt['ou_code'] == 'CN-305']
plt2['outsource_working_hour'] = plt2['outsource_working_hour'] * 100
plt2 = plt2.sort_values('operation_day')

In [ ]:
# g = go.Figure([go.Bar(x = plt2['operation_day'] ,y = plt2['inbound_receive_qty']),\
#     go.Bar(x = plt2['operation_day'], y = plt2['outbound_shipped_qty'])])
# g.update_layout(barmode='stack')
# g.add_trace(go.Scatter(x =plt2['operation_day'], y = plt2['outsource_working_hour'], mode ='lines'))
# # g.add_trace(go.Scatter(plt[plt['ou_code'] == 'CN-298'], x = 'operation_day', y =  'outsource_working_hour'))

In [ ]:
# px.box(plt2, y = ['dis_core',  'dis_core_os'])

In [ ]:
# go.Figure([go.Histogram(x = plt2['dis_core']),\
#     go.Histogram(x= plt2['dis_core_os'])])
 

In [12]:
# def percentile(n):
#     def percentile_(x):
#         return np.percentile(x, n)
#     percentile_.__name__ = 'percentile_%s' % n
#     return percentile_
def cals_cunc(df):
    cals = df.groupby(
        ['ou_code', 'kernal_core1', 'kernal_core2']
        ).agg({
            'total_working_hour': ['max', 'min', 'median', 'mean', \
                lambda a: a.quantile(.6667), 
                lambda b: b.quantile(.75)]
        }).reset_index()

    cals.columns = ['ou_code','kernal_core1','kernal_core2',\
        'max_wh','min_wh','median_wh','mean_wh','qt_66_wh','qt_75_wh']

    return cals
cals = cals_cunc(df_final)
df_final = df_final.merge(cals, on = ['ou_code', 'kernal_core1', 'kernal_core2'], how = 'left')

def mutate_cals(): 
    df_final['d_to_core_outer'] = np.abs(df_final['total_working_hour'] - df_final['kernal_value3']).round(3)
    df_final['d_to_core_outer_os'] = np.abs(df_final['outsource_working_hour'] - df_final['kernal_value4']).round(3)
    df_final['percent_error_66'] = (df_final['qt_66_wh'] - df_final['total_working_hour'])\
        /(df_final['total_working_hour'])
    df_final['percent_error_75'] = (df_final['qt_75_wh'] - df_final['total_working_hour'])\
        /(df_final['total_working_hour'])

    
    return df_final

mutate_cals()



df_final['qt_75_os'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['outsource_working_hour'].transform('quantile', .75)

df_final['pe_66_os'] = (
        df_final['qt_66_wh'] - df_final['outsource_working_hour'])/(
                df_final['outsource_working_hour']
                )
df_final['pe_75_os'] = (
        df_final['qt_75_wh'] - df_final['outsource_working_hour']
        )/(df_final['outsource_working_hour']
        )


df_final['qt_75_dis_core_os_inner'] = df_final.groupby(
    ['ou_code', 'kernal_core4']
    )['dis_core_os'].transform('quantile', .75)

df_final['qt_75_dis_core_os_outer'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['d_to_core_outer_os'].transform('quantile', .75)
# 
# df_final['flag_wh'] = [1 if df_final['dis_core_os'][i]>df_final['qt_75_dis_core_os_outer'][i]\
    #  else 0 for i in np.arange(0, len(df_final))]


# df_final['flag_75_wh'] = [1 if df_final['total_working_hour'][i]-1.2*np.abs(df_final['qt_75_wh'][i]) > 0\
#     else 0 for i in np.arange(0, len(df_final))]

# df_final['date_stamp'] = str(date.today())
# df_final['date_stamp'] = df_final['date_stamp'].str.replace('-', '')
 
# df_final['inc_day']  = '99991231'
df_final0 =df_final.replace(float('inf'), 0); df_final0

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,qt_75_wh,d_to_core_outer,d_to_core_outer_os,percent_error_66,percent_error_75,qt_75_os,pe_66_os,pe_75_os,qt_75_dis_core_os_inner,qt_75_dis_core_os_outer
0,CN-001,20210430,0.0,-1,0.000000,1699.0,2,2782.503704,32.75,1,...,45.270,0.203,0.0,0.323080,0.382290,0.0,0.0,0.0,0.0,0.0
1,CN-001,20210619,0.0,-1,0.000000,0.0,-1,0.000000,30.00,1,...,37.535,2.547,0.0,0.202337,0.251167,0.0,0.0,0.0,0.0,0.0
2,CN-001,20210721,0.0,-1,0.000000,1542.0,2,2782.503704,45.17,0,...,45.270,0.439,0.0,-0.040716,0.002214,0.0,0.0,0.0,0.0,0.0
3,CN-001,20210724,0.0,-1,0.000000,0.0,-1,0.000000,9.50,2,...,37.535,1.092,0.0,2.796854,2.951053,0.0,0.0,0.0,0.0,0.0
4,CN-001,20210811,0.0,-1,0.000000,4327.0,2,2782.503704,45.72,0,...,45.270,0.111,0.0,-0.052256,-0.009843,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8144,HK-115,20210611,34955.0,3,39025.502183,5318.0,2,6272.401310,59.10,3,...,62.160,1.049,0.0,0.046026,0.051777,0.0,0.0,0.0,0.0,0.0
8145,HK-115,20210720,58319.0,2,60992.624413,6729.0,2,6272.401310,69.37,1,...,69.370,3.364,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
8146,HK-115,20210615,65429.0,2,60992.624413,3516.0,3,2429.029316,75.55,1,...,67.445,2.816,0.0,-0.143026,-0.107280,0.0,0.0,0.0,0.0,0.0
8147,HK-115,20210830,62542.0,2,60992.624413,28971.0,1,27494.223301,84.07,4,...,84.070,2.888,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:
# df_final.groupby(
#     ['ou_code', 'kernal_core1', 'kernal_core2']
#     ).apply(lambda x: pd.Series({
#       'd_to_core_outer' : (x['total_working_hour'] - x['kernal_value3']).sum() ,
#       'd_to_core_outer_os' : x['a'].max(),
#       'd_to_core_outer_os' : x['b'].mean(),
#       'c_d_prodsum' : (x['c'] * x['d']).sum()
#   })
# )

 

In [22]:

df_final.head()
df_final['max_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('max')
df_final['min_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('min')
df_final['median_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('median')
df_final['mean_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('mean')
df_final['qt_66_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('quantile', .6667)
df_final['qt_75_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('quantile', .75)

# 直接减掉不就行了吗???? 
df_final['d_to_core_outer'] = np.abs(df_final['total_working_hour'] - df_final['kernal_value3']).round(3)

df_final['d_to_core_outer_os'] = np.abs(df_final['outsource_working_hour'] - df_final['kernal_value4']).round(3)
 

df_final['percent_error_66'] = (
        df_final['qt_66_wh'] - df_final['total_working_hour'])/(
                df_final['total_working_hour']
                )
df_final['percent_error_75'] = (
        df_final['qt_75_wh'] - df_final['total_working_hour']
        )/(df_final['total_working_hour']
        )
# 直接减掉不就行了吗???? 


df_final['qt_75_os'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['outsource_working_hour'].transform('quantile', .75)
    

df_final['pe_66_os'] = (
        df_final['qt_66_wh'] - df_final['outsource_working_hour'])/(
                df_final['outsource_working_hour']
                )
df_final['pe_75_os'] = (
        df_final['qt_75_wh'] - df_final['outsource_working_hour']
        )/(df_final['outsource_working_hour']
        )


df_final['qt_75_dis_core_os_inner'] = df_final.groupby(
    ['ou_code', 'kernal_core4']
    )['dis_core_os'].transform('quantile', .75)

df_final['qt_75_dis_core_os_outer'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['d_to_core_outer_os'].transform('quantile', .75)
# 
# df_final['flag_wh'] = [1 if df_final['dis_core_os'][i]>df_final['qt_75_dis_core_os_outer'][i]\
    #  else 0 for i in np.arange(0, len(df_final))]


# df_final['flag_75_wh'] = [1 if df_final['total_working_hour'][i]-1.2*np.abs(df_final['qt_75_wh'][i]) > 0\
#     else 0 for i in np.arange(0, len(df_final))]

# df_final['date_stamp'] = str(date.today())
# df_final['date_stamp'] = df_final['date_stamp'].str.replace('-', '')
 
# df_final['inc_day']  = '99991231'
df_final =df_final.replace(float('inf'), 0); df_final

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,qt_75_wh,d_to_core_outer,d_to_core_outer_os,percent_error_66,percent_error_75,qt_75_os,pe_66_os,pe_75_os,qt_75_dis_core_os_inner,qt_75_dis_core_os_outer
0,CN-001,20210430,0.0,-1,0.000000,1699.0,2,2782.503704,32.75,1,...,45.270,0.203,0.0,0.323080,0.382290,0.0,0.0,0.0,0.0,0.0
1,CN-001,20210619,0.0,-1,0.000000,0.0,-1,0.000000,30.00,1,...,37.535,2.547,0.0,0.202337,0.251167,0.0,0.0,0.0,0.0,0.0
2,CN-001,20210721,0.0,-1,0.000000,1542.0,2,2782.503704,45.17,0,...,45.270,0.439,0.0,-0.040716,0.002214,0.0,0.0,0.0,0.0,0.0
3,CN-001,20210724,0.0,-1,0.000000,0.0,-1,0.000000,9.50,2,...,37.535,1.092,0.0,2.796854,2.951053,0.0,0.0,0.0,0.0,0.0
4,CN-001,20210811,0.0,-1,0.000000,4327.0,2,2782.503704,45.72,0,...,45.270,0.111,0.0,-0.052256,-0.009843,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8144,HK-115,20210611,34955.0,3,39025.502183,5318.0,2,6272.401310,59.10,3,...,62.160,1.049,0.0,0.046026,0.051777,0.0,0.0,0.0,0.0,0.0
8145,HK-115,20210720,58319.0,2,60992.624413,6729.0,2,6272.401310,69.37,1,...,69.370,3.364,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
8146,HK-115,20210615,65429.0,2,60992.624413,3516.0,3,2429.029316,75.55,1,...,67.445,2.816,0.0,-0.143026,-0.107280,0.0,0.0,0.0,0.0,0.0
8147,HK-115,20210830,62542.0,2,60992.624413,28971.0,1,27494.223301,84.07,4,...,84.070,2.888,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [30]:

"""
工时异常以及额外工时标记.
"""

df_final['flag_75_os'] =  df_final['outsource_working_hour'] -1.2*df_final['qt_75_os']
df_final['flag_75_os'] = [1 if a > 0 else 0 for a in df_final['flag_75_os']]


df_final['flag_75_wh'] =  df_final['total_working_hour'] -1.2*df_final['qt_75_wh']
df_final['flag_75_wh'] = [1 if a > 0 else 0 for a in df_final['flag_75_wh']]
# df_final['flag_75_wh'] = [1 if df_final['total_working_hour'][i]\
#     -1.2*np.abs(df_final['qt_75_wh'][i]) > 0
#     else 0 for i in np.arange(0, len(df_final))]
# df_final['flag_75_wh'] = [1 if df_final['dis_core_os'][i]>np.abs(df_final['qt_75_dis_core_os_outer'][i])\
#      else 0 for i in np.arange(0, len(df_final))]

print("=================================1================================")

df_final  = df_final.replace(float('inf'), 0) 



df_final0 = df_final[df_final['flag_75_wh'] == 0] 
df_final1 = df_final[df_final['flag_75_wh'] == 1] 
df_final0['dis_tt_kernel'] = 0
df_final1['dis_tt_kernel'] = np.abs(df_final1['total_working_hour']  - 1.2* df_final1['qt_75_wh'] )


df_final = pd.concat([df_final0, df_final1], axis = 0)

print("1.2 is the final flag boundary")

df_final0 = df_final[df_final['flag_75_os'] == 0] 
df_final1 = df_final[df_final['flag_75_os'] == 1] 
df_final0['dis_os_kernel'] = 0
df_final1['dis_os_kernel'] = np.abs(df_final1['outsource_working_hour']  - 1.2* df_final1['qt_75_os'] )


df_final = pd.concat([df_final0, df_final1], axis = 0)

print("1.2 is the os flag boundary")

# diff_tt_kn = pd.concat([diff_tt_kn.rename({'qt_75_os' : 'dis_tt_kernel'}, axis = 1), \
#     df_final[df_final['flag_75_wh'] == 1]], axis = 1)[['dis_tt_kernel', 'ou_code', 'operation_day']]

# df_final = df_final.merge(diff_tt_kn, on = ['ou_code', 'operation_day'], how = 'left').fillna(0)


"""
ssr corr
"""
std_table = df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }).reset_index()

std_table.columns = ['ou_code', 'inb_qty_std', 'outb_qty_std', 'os_wh_std']

df_final = df_final.merge(std_table, on = 'ou_code', how = 'left')


def data_logs(df_final):
    df_final['log_inb_qty'] = np.log2(df_final['inbound_receive_qty'])
    df_final['log_outb_qty'] = np.log2(df_final['outbound_shipped_qty'])
    df_final_copy = df_final[
        df_final['log_inb_qty'] > -10000 & ~np.isnan(df_final['log_inb_qty'])]  
    df_final_copy2 = df_final[
        df_final['log_outb_qty'] > -10000 & ~np.isnan(df_final['log_outb_qty'])] 
    in_boundary = df_final_copy.groupby('ou_code')['log_inb_qty'].agg(['mean', 'std']).reset_index()
    ou_boundary = df_final_copy2.groupby('ou_code')['log_outb_qty'].agg(['mean', 'std']).reset_index()
    in_boundary.columns = ['ou_code', 'mean_inb_log', 'std_inb_log']
    ou_boundary.columns = ['ou_code', 'mean_oub_log', 'std_oub_log']
    return in_boundary , ou_boundary

in_boundary , ou_boundary = data_logs(df_final)
df_final = df_final.merge(
    in_boundary, on = 'ou_code', how = 'left').merge(
    ou_boundary, on = 'ou_code', how = 'left')

df_final  = df_final.replace(float('inf'), 0) 
df_final['log_inb_qty'] = df_final['log_inb_qty'].astype(float)
df_final['log_outb_qty'] = df_final['log_outb_qty'].astype(float)#log_inb_qty	log_outb_qty
print("=================================2================================")

7586     0.000
7587     0.000
7588     0.000
7589     0.000
7590     0.000
         ...  
8144     7.431
8145     7.341
8146     8.850
8147    12.850
8148     2.950
Length: 271, dtype: float64

In [27]:
# from IPython.display import display,HTML
# display(HTML(df_final0[df_final0['ou_code'] == 'CN-001'][df_final0['operation_day'] == 20210830].to_html()))

In [28]:
# pd.set_option("display.max_rows", 15, "display.max_columns", 15)
# df_final0.merge(df_final,indicator = True, how='left').loc[lambda x : x['_merge']!='both']

In [ ]:
df_final[df_final['flag_75_wh'] == 1]

In [ ]:

 
diff_tt_kn = df_final[df_final['flag_75_wh'] == 1][['total_working_hour', 'kernal_value3']].diff(
    axis = 1).drop('total_working_hour', axis = 1)
diff_tt_kn = pd.concat([diff_tt_kn.rename({'kernal_value3' : 'dis_tt_kernel'}, axis = 1), \
    df_final[df_final['flag_75_wh'] == 1]], axis = 1)[['dis_tt_kernel', 'ou_code', 'operation_day']]

df_final = df_final.merge(diff_tt_kn, on = ['ou_code', 'operation_day'], how = 'left').fillna(0)



In [ ]:
df_final

In [ ]:
df_final['log_inb_qty'] = np.log2(df_final['inbound_receive_qty'])
df_final['log_outb_qty'] = np.log2(df_final['outbound_shipped_qty'])

In [ ]:
df_final_copy = df_final[~abs(np.isinf(df_final['log_inb_qty'])) &\
     ~np.isnan(df_final['log_inb_qty'])] ; df_final_copy 
df_final_copy2 = df_final[~abs(np.isinf(df_final['log_outb_qty'])) &\
     ~np.isnan(df_final['log_outb_qty'])] ; df_final_copy2

In [ ]:

def data_logs(df_final):
    df_final['log_inb_qty'] = np.log2(df_final['inbound_receive_qty'])
    df_final['log_outb_qty'] = np.log2(df_final['outbound_shipped_qty'])
    df_final_copy = df_final[
        ~np.isinf(abs(df_final['log_inb_qty'])) & ~np.isnan(df_final['log_inb_qty'])]  
    df_final_copy2 = df_final[
        ~np.isinf(abs(df_final['log_outb_qty'])) & ~np.isnan(df_final['log_outb_qty'])] 
    in_boundary = df_final_copy.groupby('ou_code')['log_inb_qty'].agg(['mean', 'std']).reset_index()
    ou_boundary = df_final_copy2.groupby('ou_code')['log_outb_qty'].agg(['mean', 'std']).reset_index()
    in_boundary.columns = ['ou_code', 'mean_inb_log', 'std_inb_log']
    ou_boundary.columns = ['ou_code', 'mean_oub_log', 'std_oub_log']
    return in_boundary , ou_boundary


In [ ]:
# df_final_test = df_final[df_final['log_inb_qty'] < (in_boundary[0] + 2*in_boundary[1])]
# print(len(df_final_test))
# df_final_test = df_final[df_final['log_inb_qty'] > (in_boundary[0] - 2*in_boundary[1])]
# print(len(df_final_test))
in_boundary , ou_boundary = data_logs(df_final)

In [ ]:
df_final.merge(in_boundary, on = 'ou_code', how = 'left').merge(ou_boundary, on = 'ou_code', how = 'left')

In [ ]:
in_bdry_mean,in_bdry_std

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# df_final[df_final['flag_75_wh'] == 1].apply(lambda x: pd.Series({'newcol': (x['percent_error_66'] - x['percent_error_75'].sum())}))

In [ ]:
res = smf.ols('total_working_hour ~ inbound_receive_qty + outbound_shipped_qty', data=df_final).fit()

In [ ]:
print(res.summary())

In [ ]:

# df_final['percent_error_66'] = (
#         df_final['qt_66_wh'] - df_final['total_working_hour'])/(
#                 df_final['total_working_hour']
#                 )
# df_final['percent_error_75'] = (
#         df_final['qt_75_wh'] - df_final['total_working_hour']
#         )/(df_final['total_working_hour']
#         )

# # df_final = df_final.assign(percent_error_66 = [a-1 if a == 1 else a for a in df_final['percent_error_66']])
# # df_final = df_final.assign(percent_error_75 = [a-1 if a == 1 else a for a in df_final['percent_error_75']])


In [ ]:
# df_final[df_final['percent_error_75'] != 0].dropna()
from datetime import date
df_final['inc_day'] = str(date.today())
df_final['inc_day'] = df_final['inc_day'].str.replace('-', '')

In [ ]:
# df_final.to_csv('./wh_in_out_fin.csv', index= None)

In [ ]:
df_final

---

---

----

In [ ]:
tes = df_final.merge(df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }
).reset_index(), on = 'ou_code', how = 'left')


# .agg({'RECEIPT_ID':['nunique'], 
#         'TOTAL_LINES': ['sum'], 
#         'TOTAL_CONTAINERS': 'sum', 
#         'TOTAL_WEIGHT':'sum',
#         'TOTAL_QTY': ['sum'],
#         'CREATION_DATE_TIME_STAMP_2': 'min' }).reset_index()

In [ ]:
tes = df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }
).reset_index()

In [ ]:
%%time

str(date.today()).replace('-', '')

In [ ]:
df_final

In [ ]:
df_final['kernal_core1'].astype(str) + df_final['kernal_core2'].astype(str) 

In [ ]:
import plotly.express as px
 def plot3d(ou_code):
    df_plot = df_final[df_final['ou_code'] == ou_code]
    df_plot['kernal_core5'] = df_final['kernal_core1'].astype(str) + df_final['kernal_core2'].astype(str) 
    fig = px.scatter_3d(df_plot, x='inbound_receive_qty', y='outbound_shipped_qty', z='total_working_hour',
              color = 'kernal_core3')
    return fig.show()


In [ ]:
plot3d('CN-214')